In [ ]:
#Import package
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import SimpleRNN,Dense,LSTM
from keras.layers import Flatten
from sklearn.model_selection import StratifiedKFold, train_test_split, GridSearchCV
from sklearn.model_selection import GroupShuffleSplit
from sklearn import preprocessing
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error



In [ ]:
#Load pre-processed data
dataframe = pd.read_csv("timesteps=2_NeuralNetwork_new.csv")
dataframe.head()


#Split to training and testing data by proportion if testing = 0.2
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx,test_idx=next(gss.split(dataframe,groups=dataframe['stay_id']))


#get x and y variables for training and testing data
train_data=dataframe.iloc[train_idx] # data
y_train=train_data['vent_duration'] # label

x_train=train_data.drop(['Unnamed: 0','stay_id','vent_duration'],axis=1) # data
scaler=preprocessing.StandardScaler()
x_train=scaler.fit_transform(x_train)

#
test_data=dataframe.iloc[test_idx] # data
y_test=test_data['vent_duration'] # label

x_test=test_data.drop(['Unnamed: 0','stay_id','vent_duration'],axis=1) # data
scaler=preprocessing.StandardScaler()
x_test=scaler.fit_transform(x_test)


In [ ]:
#Reshape the x and y variables for both training and testing data to fit the LSTM

y_train1=y_train[::2]

x_train=x_train.reshape(-1,2,12)

x_test=x_test.reshape(-1,2,12)


In [ ]:
#Build LSTM model with 3 layers
#3 stacks LSTM layers followed by a fully-connected layer with ReLU activation function, 
#and then a finally a fully-connected alter with a linear activation function which 
#served as an output

model=Sequential()
model.add(LSTM(50,input_shape=(2,12), activation = 'relu'))
model.add(Dense(25, activation = 'relu'))
model.add(Dense(units=1,activation='linear'))

model.compile(loss='mse',optimizer='adam', metrics=['mse', 'mae'])


model.fit(x_train,y_train1, epochs=20, verbose=0)


In [ ]:
predictions=model.predict(x_test)

In [ ]:
y_test1=y_test[::2]

In [ ]:
#Model evalution


In [ ]:
# Calculate R-squared
r2 = r2_score(y_test1, predictions)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test1, predictions))

# Calculate MSE
mse = mean_squared_error(y_test1, predictions)

# Calculate MAE
mae = mean_absolute_error(y_test1, predictions)

# Print the evaluation metrics
print('R-squared:', r2)
print('RMSE:', rmse)
print('MSE:', mse)
print('MAE:', mae)